### Load Python Modules

In [ ]:
# data operations
import numpy as np
import pandas as pd

import os

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import axes3d

# For regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale
#import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf  #Provides a formula-based interface

%matplotlib inline
#plt.style.use('seaborn-white')

homedir=os.environ['HOME'] + '/'
datapath=homedir+ "datasets/"
tvsales="tvmarketing.csv"

### Read the CSV data file

In [ ]:
tvfile=f"{datapath}{tvsales}"
tv_data = pd.read_csv(tvfile)

### Data Visual

In [ ]:
# Visualise the relationship between the features and the response 
# using scatterplots
tv_data.plot(x='TV',y='Sales',kind='scatter')

### Fetch Column Names and corresponding Column Values

In [ ]:
# fetch the column headers - index 0, 1, 2 ...
xhdr=tv_data.columns[0]   # represents C0 => 'TV'
yhdr=tv_data.columns[1]   # represents C1 => 'Sales'

In [ ]:
# get TV and Sales data 
tv_col = tv_data[xhdr].values
tv_sales = tv_data[yhdr].values

### Training and Test Data

In [ ]:
# split training data based on randomness
from sklearn.model_selection import train_test_split
tv_trn, tv_tst, sales_trn, sales_tst = train_test_split(tv_col, tv_sales, \
                                                        test_size = 0.3, random_state = 1)

In [ ]:
print(f"Shapes of tv trn, tst - {tv_trn.shape}, {tv_tst.shape}")

### Reshape from 1D array to 2D array

In [ ]:
# reshape the data using array.reshape(-1, 1) as it has
# single feature - convert to 2-D array
tv_trn = tv_trn.reshape(-1, 1)
tv_tst = tv_tst.reshape(-1, 1)
print(f"Reshape of tv trn, tst - {tv_trn.shape}, {tv_tst.shape}")

In [ ]:
# import LinearRegression from sklearn
from sklearn.linear_model import LinearRegression

### Linear Regression fit()
* Goal is to minimizes the `residual sum of squares` — the distance between your actual data points and the predicted line.
* The lr.fit() expects following arguments
  - `X`: numpy array (2D) of independent variables (features)
  - `y`: numpy array (1D) of dependent variables (target)
* Calculates the optimal weights (coefficients) and bias (intercept) for a linear model. 

In [ ]:
# Representing LinearRegression as lr(Creating LinearRegression Object)
lr = LinearRegression()

# Fit the model using lr.fit(). The reshaped tv_trn, sales_trn
# are 2-D arrays - expedcted by the lr.fit()
lr.fit(tv_trn, sales_trn)

In [ ]:
# Print the intercept and coefficients
print("Intercept",lr.intercept_)
print("Slope",lr.coef_)

### Prediction using intercept and slope

In [ ]:
# Making predictions on the testing set
sales_pred = lr.predict(tv_tst)
sales_pred

### Plot the scatter - Traing data, Predicted Data

In [ ]:
plt.scatter(tv_data['TV'],tv_data['Sales'])
plt.plot(tv_trn,lr.predict(tv_trn),color='red')
plt.xlabel('TV')
plt.ylabel('Sales')

### Visual of Observed versus Predicted

In [ ]:
# Actual vs Predicted
import matplotlib.pyplot as plt
c = [i for i in range(1,61,1)]         # generating index 
fig = plt.figure()
plt.plot(c, sales_tst, color="navy", linewidth=2, linestyle="-")
plt.plot(c, sales_pred, color="red", linewidth=2, linestyle="-")
fig.suptitle('Actual and Predicted using Scikitlearn', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                               # X-label
plt.ylabel('Sales', fontsize=16) 

### Error Metrics
* Residual calculation ( epsilon = y_test - y_pred) 
* Used as the building block for common performance metrics
  - Mean Squared Error (MSE)
  - Root Mean Squared Error (RMSE). 

In [ ]:
c = [i for i in range(1,61,1)]
fig = plt.figure()
plt.plot(c, sales_tst - sales_pred, color="blue", linewidth=2, linestyle="-")
fig.suptitle('Error Terms', fontsize=20)              # Plot heading 
plt.xlabel('Index', fontsize=18)                      # X-label
plt.ylabel('sales_tst - sales_pred', fontsize=16)                # Y-label

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(sales_tst, sales_pred)
r_squared = r2_score(sales_tst, sales_pred)
print(f"mean sq error {mse}, r2 score {r_squared}")

## Visualizing Coefficients

### 1D array of equally spaced points using bias b (x) and slope m (y)

In [ ]:
b = np.linspace(lr.intercept_-2, lr.intercept_+2, 50)
m = np.linspace(lr.coef_-0.02, lr.coef_+0.02, 50)

X = tv_trn
Y = sales_trn

In [ ]:
print(type(b), type(m))
print(b.size, m.size)

### Calculate the Residual Sum of Squares

In [ ]:
xx, yy = np.meshgrid(b, m, indexing='xy')
Z = np.zeros((b.size,m.size))

# method 1 to calculate Sigma of ( Y - (MX + B) ) squared
for i in range(b.size):
    for j in range(m.size):
        Z[j,i] = np.sum((Y - (m[j]*X + b[i]))**2)

### This method needs to be verified

In [ ]:
# method 2
# Z = np.zeros((b.size,m.size))
# for (i,j),v in np.ndenumerate(Z):
#     Z[i,j] = np.sum((Y - (xx[i,j]+X.ravel()*yy[i,j]))**2).sum()

In [ ]:
# Minimized RSS
min_RSS = r'b, m for minimized RSS'
min_rss = np.sum((lr.intercept_+lr.coef_*X - Y.reshape(-1,1))**2)
min_rss

### Plot the data
* Scatter of `intercepts` and `slopes`
* TBD - understand the scatter, plotting diagrams etc.

In [ ]:
fig = plt.figure(figsize=(15,6))
fig.suptitle('RSS - Regression coefficients', fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, projection='3d')

# Left plot
#contour plot on the first subplot (ax1) with x-coordinates xx,
#y-coordinates yy and RSS values Z, using the colormap plt.cm.Set1.
#inline=True: Specifies that the contour labels should be placed inside the contour lines.
CS = ax1.contour(xx, yy, Z, cmap=plt.cm.Set1)
ax1.scatter(lr.intercept_, lr.coef_[0], c='r', label=min_RSS)
ax1.clabel(CS, inline=True, fontsize=10, fmt='%1.1f')

# Right plot
#surface plot on the second subplot (ax2) with x-coordinates xx, 
#y-coordinates yy and RSS values Z, using the colormap plt.cm.Set1.
#rstride- An integer value specifying the stride used to sample the rows of Z.
#cstride-  An integer value specifying the stride used to sample the columns of Z
ax2.plot_surface(xx, yy, Z, rstride=3, cstride=3, alpha=0.3)
ax2.contour(xx, yy, Z, zdir='z', offset=Z.min(), cmap=plt.cm.Set1,
            alpha=0.4)
ax2.scatter3D(lr.intercept_, lr.coef_[0], min_rss, c='r', label=min_RSS)
ax2.set_zlabel('RSS')
#ax2.set_zlim(Z.min(),Z.max())
#ax2.set_ylim(0.02,0.07)

# settings common to both plots
for ax in fig.axes:
    ax.set_xlabel(r'b', fontsize=17)
    ax.set_ylabel(r'm', fontsize=17)
   # ax.set_yticks([0.03,0.04,0.05,0.06])
    ax.legend()

## Example - usage of matlibplot, contour etc.
* Enable the cell to see how it works

In [ ]:
cell_enabled = False
if cell_enabled:
    import matplotlib.pyplot as plt
    import numpy as np

    # 1. Generate sample data (for a 3D surface)
    x = np.linspace(-2, 2, 100)
    y = np.linspace(-2, 2, 100)
    XT, YT = np.meshgrid(x, y) # Create a grid of points
    ZT = np.sin(XT) * np.cos(YT) # Calculate corresponding Z values

    # 2. Create a figure and add a subplot
    fig = plt.figure(figsize=(10, 4))
    ax1 = fig.add_subplot(1, 2, 1) # First subplot (1 row, 2 columns, index 1)
    ax2 = fig.add_subplot(1, 2, 2) # Second subplot (1 row, 2 columns, index 2)

    # 3. Plot a contour on the first subplot (ax1)
    ax1.contour(XT, YT, ZT, levels=10, cmap='viridis') # Use the ax1.contour() method
    ax1.set_title('Contour Plot on ax1')
    ax1.set_xlabel('X-axis')
    ax1.set_ylabel('Y-axis')

    # 4. Plot a filled contour on the second subplot (ax2)
    ax2.contourf(XT, YT, ZT, levels=20, cmap='coolwarm') # Use the ax2.contourf() method
    ax2.set_title('Filled Contour Plot on ax2')
    ax2.set_xlabel('X-axis')
    ax2.set_ylabel('Y-axis')

    # Adjust layout and display
    plt.tight_layout()
    plt.show()


### Quick basics check

In [ ]:
a = [ 1, 2, 3 ]
b = [ 10, 20, 30 ]
print(f"a -> {a}, b -> {b}")
print(f"{type(a)}, {type(b)}")

In [ ]:
npa = np.array(a)
npb = np.array(b)
print(f"npa -> {npa}, npb -> {npb}")
print(f"types -> {type(npa)}, {type(npb)}")

In [ ]:
print(f"shapes -> {npa.shape}, {npb.shape}")

In [ ]:
r_npa = npa.reshape(-1, 1)
r_npa

In [ ]:
r_npb = npb.reshape(1, -1)
r_npb

In [ ]:
npc = r_npa * r_npb
print(f"type -> {type(npc)}")
npc

## Using STATS models
* Calculate following
  - `Coefficients`
  - `Standard Error`
  - `t-value`
  - `p-value`

In [ ]:
import statsmodels.formula.api as smf
est1 = smf.ols('Sales ~ TV', tv_data).fit()   #sales is modeled as a function of TV
est1.summary().tables[1]

### More STATS model data 

In [ ]:
est1.summary()

## Manual Calculation of `t-value`, `p-value`

In [ ]:
# reshape TV column to 200 (row) X 1 (col)
x = tv_col.reshape(-1,1)
# Sales column remains the same 1 (row) X 200 (col)
y = tv_sales


In [ ]:
# Shapes are 
# x : 2D array - matrix with 200 rows, 1 column
# y : 1D array - list of 200 items 
#     (behaves like 1 row, 200 columns)
print(f"Shapes -> x {x.shape}, y {y.shape}")

### Calculate weights (coefficients) and bias (intercept)

In [ ]:
lreg = LinearRegression()
model = lreg.fit(x,y)
#lreg.coef_, lreg.intercept_
model

### Calculate `residuals` (y_i - y_i_pred)

In [ ]:
y_pred = model.predict(x)
residuals = y - y_pred
residuals

### Degrees of Freedom in MLR model

In [ ]:
# number of observations
n = len(x)
# number of predictors
k = x.shape[1]
# degrees of freedom of residuals in MLR model.
df_residuals = n - k - 1
print(f"n -> {n}, k -> {k}, df -> {df_residuals}")

### Calculate the following
* `SSE` - Sum of Squared Errors
* `MSE` - Mean Squared Error

In [ ]:
# calculate the SSE (Sum of Squared Errors)
sigma_sse = np.sum(residuals ** 2)
# calculate the residual variance - Mean Square Error (MSE)
residual_variance = sigma_sse / df_residuals
sigma_sse

### Variance-Covariance Matrix 
* Contains `variances` of each coefficient on the `diagonal` 
* Contains `covariances` between coefficients `off-diagonal`. 
* Scaling by the MSE adjusts the values to represent the variances and covariances in the regression model.

In [ ]:
# Create 2D numpy array of shape (200,1) all filled 
# with 1. (float64)
x_ones = np.ones((len(x), 1))
x_ones.shape

### Quick check basics
* np.hstack() function example
* `a` is 2 X 2 matrix, `b` is 2 X 1 matrix
* `np.hstack()` concatenates the `a`, `b` along the `columns`
* `number of rows` must be same for the provided matrices

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[8], [9]])
print(a)
print(b)

In [ ]:
c = np.hstack([a, b])
c

### Matrix Operations

In [ ]:
# Transpose
at = a.T
at

In [ ]:
# product and dot product
p1 = a * at
p2 = np.dot(a, at)
print(f"p1 -> {p1}")
print(f"p2 -> {p2}")

### Variance, Co-variance matrix of the Coefficients

In [ ]:
# hstack operation
X = np.hstack([x_ones, x])

In [ ]:
# inv(X^T dot X)
XTX_inv = np.linalg.inv(np.dot(X.T, X))
print(f"inv(X^T dot X) -> {XTX_inv}")
print(f"Shape -> {XTX_inv.shape}")

In [ ]:
# Scale the inverse matrix by the residual variance to 
# get the variance-covariance matrix of the coefficients
var_cov_matrix = sigma_sse * XTX_inv
print("Variance Covariance Matrix")
print(var_cov_matrix)

In [ ]:
# Extract standard error of the intercept (first diagonal 
# element)
SE_intercept = np.sqrt(var_cov_matrix[0, 0])

print("Standard error of the intercept:", SE_intercept)

# Calculate estimated value of the intercept
intercept = model.intercept_

# Calculate t-value for the intercept
t_value_intercept = intercept / SE_intercept

print("t-value for the intercept:", t_value_intercept)

from scipy.stats import t

# Calculate degrees of freedom
df = n - k - 1  # Same as df_residual

# Calculate p-value for the intercept
p_value_intercept = 2 * (1 - t.cdf(np.abs(t_value_intercept), df))

print("p-value for the intercept:", p_value_intercept)